In [1]:
import datetime
import podpac
from podpac.datalib import smap_egi
import xarray as xr
import zarr

C:\Anaconda3\envs\podpac\lib\site-packages\xarray\core\merge.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [2]:
bucket = "podpac-internal-test"
source = 's3://%s/%s' % (bucket, 'SPL3SMP_E.002_9km_aid0001.zarr')
source2 = 's3://%s/%s' % (bucket, 'SPL4SMGP.004_9km_aid0001.zarr')

kwargs = {'dims': ['time', 'lat', 'lon'], 'latkey': 'fakedim0', 'lonkey': 'fakedim1',
          'cf_time': True, 'cf_calendar': 'proleptic_gregorian', 'cf_units': 'days since 2000-01-01',
         'crs': 'epsg:6933', 'nan_vals': [-9999]}
smap_zarr = podpac.data.Zarr(source=source, datakey='Soil_Moisture_Retrieval_Data_AM_soil_moisture', file_mode='a', **kwargs)
smap_ds = smap_zarr.dataset

smap_zarr_L4 = podpac.data.Zarr(source=source2, datakey='Geophysical_Data_sm_surface', file_mode='a', **kwargs)
smap_ds_L4 = smap_zarr_L4.dataset

In [4]:
coords = smap_zarr.native_coordinates.drop('time')
time = podpac.crange(smap_zarr.native_coordinates['time'].bounds[-1], str(datetime.date.today()), '1,D', 'time')
coords = podpac.coordinates.merge_dims([podpac.Coordinates([time], crs=coords.crs), coords])
coords

Coordinates (epsg:6933)
	time: UniformCoordinates1d(time): Bounds[2019-06-20T00:00:00.000000000, 2019-11-22T00:00:00.000000000], N[156], ctype['midpoint']
	lat: ArrayCoordinates1d(lat): Bounds[2878072.923230057, 5589497.541484003], N[302], ctype['midpoint']
	lon: ArrayCoordinates1d(lon): Bounds[-12174386.616512204, -6436255.447649201], N[638], ctype['midpoint']

In [5]:
coords_l4 = smap_zarr_L4.native_coordinates.drop('time')
time = podpac.crange(smap_zarr_L4.native_coordinates['time'].bounds[-1], str(datetime.date.today()), '1,D', 'time')
coords_l4 = podpac.coordinates.merge_dims([podpac.Coordinates([time],  crs=coords_l4.crs), coords_l4])
coords_l4

Coordinates (epsg:6933)
	time: UniformCoordinates1d(time): Bounds[2019-07-10T00:00:00.000000000, 2019-11-22T00:00:00.000000000], N[136], ctype['midpoint']
	lat: ArrayCoordinates1d(lat): Bounds[2878072.923230057, 5589497.541484003], N[302], ctype['midpoint']
	lon: ArrayCoordinates1d(lon): Bounds[-12174386.616512204, -6436255.447649201], N[638], ctype['midpoint']

In [ ]:
# Get all the new data
# L4 data
smap = smap_egi.SMAP(product='SPL4SMAU')
l4 = smap.eval(coords_l4)

# AM Data
smap = smap_egi.SMAP(product='SPL3SMP_E_AM', check_quality_flags=False)
l3_am = smap.eval(coords)
smap.set_trait('data_key', smap.quality_flag_key)
l3_am_qf = smap.eval(coords)

# PM Data
smap = smap_egi.SMAP(product='SPL3SMP_E_PM', check_quality_flags=False)
l3_pm = smap.eval(coords)
smap.set_trait('data_key', smap.quality_flag_key)
l3_pm_qf = smap.eval(coords)



c:\repositories\pipeline\podpac\podpac\core\coordinates\dependent_coordinates.py:496: UserWarning: transformation of coordinate segment lengths not yet implemented
  warnings.warn("transformation of coordinate segment lengths not yet implemented")
c:\repositories\pipeline\podpac\podpac\core\coordinates\dependent_coordinates.py:498: UserWarning: transformation of coordinate segment lengths not yet implemented
  warnings.warn("transformation of coordinate segment lengths not yet implemented")


In [ ]:
# replace_nans
l4.set(-9999, np.isnan(l4))
l3_am.set(-9999, np.isnan(l3_am))
l3_am_qf.set(-9999, np.isnan(l3_am_qf))
l3_pm.set(-9999, np.isnan(l3_pm))
l3_pm_qf.set(-9999, np.isnan(l3_pm_qf))


In [18]:
time_base = xr.coding.times.decode_cf_datetime(0, smap_ds['time'].attrs['units'])
new_times = (coords.coords['time'].data - time_base).astype('timedelta64[D]')[1:]
new_times

array([7111], dtype='timedelta64[D]')

In [69]:
# Update the zarr file
## IMPORTANT ONLY EXECUTE THIS CELL ONCE!!!! ANY ERRORS? DO NOT EXECUTE THE SAME APPEND AGAIN!
smap_ds['time'].append(new_times)
smap_ds['Soil_Moisture_Retrieval_Data_AM_soil_moisture'].append(l3_am[1:], axis=0)
smap_ds['Soil_Moisture_Retrieval_Data_AM_retrieval_qual_flag'].append(l3_am_qf[1:], axis=0)
smap_ds['Soil_Moisture_Retrieval_Data_PM_soil_moisture_pm'].append(l3_pm[1:], axis=0)
smap_ds['Soil_Moisture_Retrieval_Data_PM_retrieval_qual_flag_pm'].append(l3_pm_qf[1:], axis=0)

(112, 302, 638)

In [23]:
time_base = xr.coding.times.decode_cf_datetime(0, smap_ds_L4['time'].attrs['units'])
new_times = (coords_l4.coords['time'].data - time_base).astype('timedelta64[D]')
new_times

array([7129, 7130], dtype='timedelta64[D]')

In [31]:
# Update the zarr file
## IMPORTANT ONLY EXECUTE THIS CELL ONCE!!!! ANY ERRORS? DO NOT EXECUTE THE SAME APPEND AGAIN!
smap_ds_L4['time'].append(new_times[1:])
smap_ds_L4['Geophysical_Data_sm_surface'].append(l4[1:], axis=0

(127, 302, 638)